# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import library
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sqlalchemy import create_engine
%matplotlib inline

import pprint as pp

# import ml library
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# import pri library
import wrangling3 as w

# set env
## show all row and column
from IPython.display import display
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
## show 200 string in context
pd.set_option('max_colwidth',200)

In [6]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',con=engine)
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [8]:
X = df['message']
Y = df[df.columns[5:]]
X.sample()

18936    Niger firmly condemned human rights violations committed by Boko Haram and said that this terrorist group had not yet been beaten and continued to raid villages and massacre civilians with unprece...
Name: message, dtype: object

In [9]:
Y.sample()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
14435,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [11]:
#nltk.download('wordnet')

In [12]:
tokenize(X[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
# arc for init test
'''
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])
'''

In [43]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [44]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X = df.message
X.head(1)

0    Weather update - a cold front from Cuba that could pass over Haiti
Name: message, dtype: object

In [15]:
y = df[df.columns[5:]]
#Y = df.loc[:,df.columns[4:]]
y.head(1)

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
y = y.astype(float)
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 35 columns):
request                   26216 non-null float64
offer                     26216 non-null float64
aid_related               26216 non-null float64
medical_help              26216 non-null float64
medical_products          26216 non-null float64
search_and_rescue         26216 non-null float64
security                  26216 non-null float64
military                  26216 non-null float64
child_alone               26216 non-null float64
water                     26216 non-null float64
food                      26216 non-null float64
shelter                   26216 non-null float64
clothing                  26216 non-null float64
money                     26216 non-null float64
missing_people            26216 non-null float64
refugees                  26216 non-null float64
death                     26216 non-null float64
other_aid                 26216 non-null float64
infrastru

In [ ]:
import wrangling3 as w
#w.checkvalue(y)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [45]:
model = pipeline

In [46]:
model.fit(X_train, y_train);

/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [47]:
y_pred = model.predict(X_test)

In [48]:
cols = df.columns[5:]

In [49]:
y_pred = pd.DataFrame(y_pred,columns=cols)

In [50]:
y_pred.sample()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
121,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [51]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6554 entries, 16025 to 20741
Data columns (total 35 columns):
request                   6554 non-null float64
offer                     6554 non-null float64
aid_related               6554 non-null float64
medical_help              6554 non-null float64
medical_products          6554 non-null float64
search_and_rescue         6554 non-null float64
security                  6554 non-null float64
military                  6554 non-null float64
child_alone               6554 non-null float64
water                     6554 non-null float64
food                      6554 non-null float64
shelter                   6554 non-null float64
clothing                  6554 non-null float64
money                     6554 non-null float64
missing_people            6554 non-null float64
refugees                  6554 non-null float64
death                     6554 non-null float64
other_aid                 6554 non-null float64
infrastructure_related  

In [52]:
print(classification_report(y_test['offer'],y_pred['offer']))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6529
         1.0       0.00      0.00      0.00        25

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554



/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
for i in cols:
    print('- - - {} - - -'.format(i))
    print(classification_report(y_test[i],y_pred[i]))

- - - request - - -
              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93      5441
         1.0       0.83      0.36      0.50      1113

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.67      0.72      6554
weighted avg       0.87      0.88      0.86      6554

- - - offer - - -
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6529
         1.0       0.00      0.00      0.00        25

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

- - - aid_related - - -
              precision    recall  f1-score   support

         0.0       0.69      0.91      0.79      3881
         1.0       0.76      0.41      0.53      2673

   micro avg       0.71      0.71      0.71      6554
   macro avg       0.72      0.66      0.66      6554
weighted avg

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6247
         1.0       0.71      0.02      0.03       307

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.83      0.51      0.50      6554
weighted avg       0.94      0.95      0.93      6554

- - - direct_report - - -
              precision    recall  f1-score   support

         0.0       0.85      0.98      0.91      5298
         1.0       0.78      0.28      0.42      1256

   micro avg       0.85      0.85      0.85      6554
   macro avg       0.82      0.63      0.66      6554
weighted avg       0.84      0.85      0.82      6554



In [54]:
# archive
# 如果不转换 y_pred 需要进行转换后比较
# print(classification_report(y_test.iloc[:,0], y_pred.T[0]))；

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# slow lap, take small featrues to run grid
'''
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }
'''

In [59]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

    # specify parameters for grid search
    parameters = {
        'features__text_pipeline__vect__max_df': (0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100],
        'clf__min_samples_split': [3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }


    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

SyntaxError: invalid syntax (<ipython-input-59-2047cd9c21a5>, line 16)

In [58]:
model = build_model()
model.fit(X_train, y_train)

/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/francis/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.